In [4]:
import requests
import xml.etree.ElementTree as ET

def get_exchange_data():
    """Fetch and parse the daily ECB XML feed."""
    url = "https://www.ecb.europa.eu/stats/eurofxref/eurofxref-daily.xml"
    try:
        response = requests.get(url)
        response.raise_for_status()
        root = ET.fromstring(response.content)
        
        # ECB uses a specific namespace
        ns = {'ex': 'http://www.ecb.int/vocabulary/2002-08-01/eurofxref'}
        
        # Build a dictionary of rates { 'USD': 1.08, 'GBP': 0.83, ... }
        # We add 'EUR': 1.0 manually since it's the anchor
        rates = {'EUR': 1.0}
        for cube in root.findall('.//ex:Cube[@currency]', ns):
            rates[cube.get('currency')] = float(cube.get('rate'))
            
        return rates
    except Exception as e:
        print(f"Error: {e}")
        return None

def convert_currency(amount, from_code, to_code, rates):
    """Calculate the conversion between any two currencies in the feed."""
    if from_code not in rates or to_code not in rates:
        return None
    
    # Logic: (Amount / Base_Rate_vs_EUR) * Target_Rate_vs_EUR
    eur_value = amount / rates[from_code]
    target_value = eur_value * rates[to_code]
    return target_value

# --- Execution ---
rates_dict = get_exchange_data()

if rates_dict:
    print("ECB Daily Rates Loaded.")
    base = input("Convert FROM (e.g. USD): ").upper().strip()
    target = input("Convert TO (e.g. GBP): ").upper().strip()
    amt = float(input(f"Amount of {base}: "))
    
    result = convert_currency(amt, base, target, rates_dict)
    
    if result:
        print(f"\n{amt} {base} = {result:.4f} {target}")
        print(f"Rate: 1 {base} = {(result/amt):.4f} {target}")
    else:
        print("One or both currency codes were not found.")

ECB Daily Rates Loaded.


Convert FROM (e.g. USD):  USD
Convert TO (e.g. GBP):  JPY
Amount of USD:  1



1.0 USD = 157.3180 JPY
Rate: 1 USD = 157.3180 JPY
